In [ ]:
# -*- coding: UTF8 -*-

import sys
import io
import os
import re
import csv
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

# Selenium WebDriver로 Chrome 실행
driver = webdriver.Chrome()

for page_number in range(1, 45):  # 1부터 44까지의 페이지를 반복
    url = f"https://www.rallit.com/hub?pageNumber={page_number}"
    driver.get(url)
    time.sleep(5)

    # 결과 저장 리스트
    result_list = []

    # 각 페이지의 요소를 클릭하여 데이터 수집
    for i in range(1, 21):
        xpath = f'//*[@id="hub-container"]/section[2]/ul/li[{i}]/a/article/div'
        element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath)))
        element.click()
        
        
        section_ids = {
            "직업": "basic-section",
            "기술 스택": "techStack-section",
            "경력": "workExperiences-section",
            "프로젝트": "projects-section",
            "대외활동": "activities-section",
            "교육": "educations-section",
            "자격증": "certificates-section",
            "외국어": "foreignLanguages-section"
        }

        # 각 섹션의 데이터 수집
        result = {}
        for section_name, section_id in section_ids.items():
            if section_name == "직업":
                try:
                    section_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, section_id)))
                    job_element = section_element.find_element(By.XPATH, "//div[dt='직업']/dd")
                    job_text = job_element.text
                    result[section_name] = job_text
                except Exception as e:
                    result[section_name] = ""
            else:
                try:
                    section_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, section_id)))
                    section_texts = section_element.find_elements(By.CLASS_NAME, 'preview-description')
                    section_texts = [text.text for text in section_texts]
                    result[section_name] = ', '.join(section_texts)
                except Exception as e:
                    result[section_name] = ""

        result_list.append(result)
        print(result)
        driver.back()  # 이전 페이지로 이동
        
# WebDriver 종료
driver.quit()

print(result_list)        

In [ ]:
import pandas as pd

# result_list를 데이터프레임으로 변환
df = pd.DataFrame(result_list)
df